In [ ]:
# 📌 Install Dependencies
!pip install diffusers transformers accelerate torch deep_translator faiss-cpu sentence-transformers --quiet

Import libraries and download data from Google Drive

In [ ]:
# 📌 Import Libraries
import torch
import faiss
import pandas as pd
from deep_translator import GoogleTranslator
from diffusers import StableDiffusionPipeline
from sentence_transformers import SentenceTransformer
import gdown
import os

# 📌 Define Paths for Download
FAISS_INDEX_URL = "https://drive.google.com/uc?id=1flF3u7iy4kzi58GBZnNEz5H11A9Rny4h"
PROCESSED_DATASET_URL = "https://drive.google.com/uc?id=1eXA_cnZkuQCafWSBI1hh_naPdnXVDC0R"

FAISS_INDEX_PATH = "/content/faiss_index.bin"
PROCESSED_DATASET_PATH = "/content/processed_dataset.csv"

# 📌 Download Data from Google Drive
if not os.path.exists(FAISS_INDEX_PATH):
    print("📥 Downloading FAISS index...")
    gdown.download(FAISS_INDEX_URL, FAISS_INDEX_PATH, quiet=False)

if not os.path.exists(PROCESSED_DATASET_PATH):
    print("📥 Downloading processed dataset...")
    gdown.download(PROCESSED_DATASET_URL, PROCESSED_DATASET_PATH, quiet=False)

Setting up

In [ ]:
# 📌 Load Data
print("📂 Loading dataset...")
df = pd.read_csv(PROCESSED_DATASET_PATH)
print("✅ Dataset loaded!")

# 📌 Load Model
print("📥 Loading sentence transformer model...")
model = SentenceTransformer("jonny9f/food_embeddings", device="cuda" if torch.cuda.is_available() else "cpu")
print("✅ Model loaded!")

# 📌 Load FAISS Index
print("📥 Loading FAISS index...")
faiss_index = faiss.read_index(FAISS_INDEX_PATH)
print("✅ FAISS index loaded!")

# 📌 Load Stable Diffusion for Image Generation
print("📥 Loading Stable Diffusion...")
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)
print("✅ Stable Diffusion loaded!")

Defining Image Generation function

In [ ]:
# 📌 Define Search and Image Generation Function
def generate_recipe_image(recipe_name):
    """ Generate a new image based on the recipe name using Stable Diffusion."""
    print(f"🎨 Generating image for: {recipe_name}")
    prompt = f"A delicious dish of {recipe_name}, beautifully plated, high-quality food photography."

    with torch.autocast(device):
        image = pipe(prompt).images[0]

    return image

Run the application

In [ ]:
from IPython.display import display
# 📌 Run the Recipe Search
lang_options = {"en": "English", "pt": "Português", "es": "Español"}
print("\n🌍 Choose display language: ")
for code, name in lang_options.items():
    print(f"  [{code}] {name}")

selected_lang = input("\nEnter the desired language code (en, pt, es): ").strip().lower()
if selected_lang not in lang_options:
    print("❌ Invalid language! Defaulting to English.")
    selected_lang = "en"
print(f"\n✅ Language selected: {lang_options[selected_lang]}")

query_text = input("\n🔍 Enter the dish name or description you want to search for: ").strip()
query_embedding = model.encode([query_text], convert_to_numpy=True)

k = 3  # Number of recommendations
distances, indices = faiss_index.search(query_embedding, k)

print(f"\n🔍 **Results for:** {query_text} ({lang_options[selected_lang]})")

for i, idx in enumerate(indices[0]):
    recipe = df.iloc[idx]
    title = recipe["title"]
    ingredients = recipe["ingredients"]
    directions = recipe["directions"]

    if selected_lang != "en":
        title = GoogleTranslator(source="en", target=selected_lang).translate(title)
        ingredients = GoogleTranslator(source="en", target=selected_lang).translate(ingredients)
        directions = GoogleTranslator(source="en", target=selected_lang).translate(directions)

    print(f"\n🔹 **Recommendation {i+1}**")
    print(f"🍽️ Recipe: {title}")
    print(f"🛒 Ingredients: {ingredients}")
    print(f"📖 Instructions: {directions}")
    print(f"\n⚡ **FAISS Distance:** {distances[0][i]}")
    print("-" * 80)

    image = generate_recipe_image(title)
    display(image)
